In [ ]:
# https://gist.github.com/sourangshupal/54e8702ddcba446bac603dd8908e6ce1

In [1]:
!pip install transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 173.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 176.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 206.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 241.1 MB/s eta 0:00:0000:01


In [5]:
!pip install langchain langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 146.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 186.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.6/580.6 kB 45.8 MB/s eta 0:00:00
  Attempting uninstall: marshmallow
    Found existing installation: marshmallow 4.0.0
    Uninstalling marshmallow-4.0.0:
      Successfully uninstalled marshmallow-4.0.0


In [ ]:
# https://huggingface.co/docs/transformers/v4.51.3/en/main_classes/pipelines

In [2]:
from transformers import pipeline
import torch
import base64
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# https://huggingface.co/MBZUAI/LaMini-T5-738M
# SLM (Small language model)
checkpoint = "MBZUAI/LaMini-T5-738M"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# # https://huggingface.co/docs/transformers/v4.51.3/en/main_classes/pipelines
from langchain_huggingface import HuggingFacePipeline
def slm_pipeline():
    pipe = pipeline(
        "text2text-generation",
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_slm = HuggingFacePipeline(pipeline = pipe)
    return local_slm

In [10]:
input_prompt = "Write an article about Blockchain and its benefits"

In [12]:
model = slm_pipeline()
gen_text = model.invoke(input_prompt)
gen_text

Device set to use cpu


'Blockchain is a decentralized digital ledger that records transactions in a secure and transparent manner. It is a decentralized system that allows for secure and transparent transactions without the need for intermediaries. One of the most significant benefits of blockchain is its ability to eliminate the need for intermediaries. Blockchain technology eliminates the need for intermediaries, allowing for a decentralized and transparent system. This means that transactions can be recorded on the blockchain without the need for intermediaries. This makes it a valuable tool for businesses and governments to track and verify transactions, reducing the risk of fraud and ensuring transparency. Another'

In [13]:
# Deployment
# For below code goto https://huggingface.co/MBZUAI/LaMini-T5-738M  >goto deploy> copy-paste code here>make changes as done in below code
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
	'HF_TASK' : 'text2text-generation',
    'device_map' : 'auto',
    'torch_dtype' : 'torch.float32'
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.2.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Write an article about Cyber Security",
})

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/11/25 09:43:28] INFO     Found credentials from IAM Role:                                   ]8;id=165494;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=736948;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[05/11/25 09:43:30] INFO     Found credentials from IAM Role:                                   ]8;id=417528;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=896262;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=867420;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=511882;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

...

[05/11/25 09:43:31] INFO     Creating model with name:                                              ]8;id=730176;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=483014;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-2025-05-11-09-43-31-037                             

                    INFO     Creating endpoint-config with name                                     ]8;id=987062;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=138516;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             huggingface-pytorch-tgi-inference-2025-05-11-09-43-31-576                             

                    INFO     Creating endpoint with name                                            ]8;id=444669;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=711923;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\
                             huggingface-pytorch-tgi-inference-2025-05-11-09-43-31-576                             

-------------!

[{'generated_text': 'Cybersecurity is the practice of protecting digital systems from unauthorized access, use, theft, or damage. This includes utilizing various cybersecurity strategies, as well as educating businesses and individuals on how to prevent and detect cyber threats. Secure storage and access control systems are crucial in securing sensitive data and equipment from unauthorized access or destruction. Aside from physical security measures, such as strong passwords and multi-factor authentication, secure computing and storage options are essential as well. Additionally, outsourcing security roles, providing training on security best practices, and implementing robust risk assessment and vulnerability assessment can also help in raising'}]

In [18]:
# To get this endpoint goto 'AWS Sagemaker AI'> Under 'Inference' click on 'Endpoints'> copy the endpoint present uner 'Name' and paste here
# One more option to get endpoint is endpoint is output of above cell (last line)
ENDPOINT = "huggingface-pytorch-tgi-inference-2025-05-11-09-43-31-576"

In [20]:
import boto3

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

endpoint_name = ENDPOINT

In [21]:
# API Payload
prompt = "Write an article on Deep learning"

payload = {
    'inputs' : prompt ,
    'parameters' : {
        'max_new_tokens' : 256,
        'do_sample' : True,
        'temperature' : 0.3,
        'top_p' : 0.7,
        'top_k' : 50,
        'repetion_penalty' : 1.03
    }
}

response = sagemaker_runtime.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = "application/json",
    Body = json.dumps(payload)
)

In [23]:
response

{'ResponseMetadata': {'RequestId': 'b2f54dfb-7dec-42d2-816b-f711d71b8527',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b2f54dfb-7dec-42d2-816b-f711d71b8527',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sun, 11 May 2025 10:11:20 GMT',
   'content-type': 'application/json',
   'content-length': '709',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f1bbf18fc70>}

In [22]:
predictions = json.loads(response['Body'].read().decode('utf-8'))
final_result =predictions[0]['generated_text']

In [24]:
predictions

[{'generated_text': 'Deep learning is a subset of machine learning that uses artificial neural networks to analyze and learn from large amounts of data. It is a type of machine learning that uses algorithms to recognize patterns and make predictions from data. Deep learning is a popular technique used in various industries, including finance, healthcare, and transportation. It involves training a model on a large dataset of labeled data to identify patterns and make predictions. One of the most significant applications of deep learning is in image and speech recognition. Deep learning algorithms can be used to analyze large amounts of data, such as facial recognition, facial recognition, and speech'}]